In [ ]:
%cd ../src
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
requests.packages.urllib3.disable_warnings() 

import claim_verification

In [ ]:
# if torch.cuda.is_available():    
#     # Tell PyTorch to use the GPU.    
#     device = torch.device("cuda")
#     print('There are %d GPU(s) available. {}'.format(torch.cuda.device_count()))
#     print('We will use the GPU: {}'.format(torch.cuda.get_device_name(0)))

# # If we dont have GPU but a CPU, training will take place on CPU instead
# else:
#     print('No GPU available, using the CPU instead.')
#     device = torch.device("cpu")

In [ ]:
def flatten_evidence(parsed_evidence):
    if type(parsed_evidence) == str:
        return parsed_evidence
    else:
        return ' '.join([' '.join([' '.join([el if len(el)>2 else '' for el in sl]).strip() for sl in lst]) for lst in parsed_evidence]).strip()

In [ ]:
def summarize_text(text):
    global summarizer
    
    # truncate input somewhat (due to model max length)
    # TODO split context into chunks of max len, concat later?
    maxlen = 700
    if type(text) == str:
        text = ' '.join([txt for txt in text.split(' ')[:maxlen]])
    else:
        text = [' '.join([txt for txt in subtext.split(' ')[:maxlen]]) for subtext in text]

        summarized_text = summarizer(text, min_length=min(10, len(text)), max_length=(min(128, len(text))))
    
    if type(text) != str:
        summarized_text = [txt['summary_text'] for txt in summarized_text]
    else:
        summarized_text = summarized_text[0]['summary_text']
        
    return summarized_text

In [ ]:
def collect_evidence(claim, num_results):
    try:
        evidence = load_evidence(claim)
        print('Existing evidence results found locally, loading from disk.')
    except:
        links = get_evidences.get_top_k_results_from_google(claim, k=num_results)
        evidence = [(l, flatten_evidence(get_evidences.get_relevant_text_from_webpage(l))) for l in links]

        # Remove empty evidence
        evidence = pd.DataFrame([ev for ev in evidence if len(ev[1]) > 0], columns=['source', 'text'])
        
        # Clean input
        evidence['text'] = evidence['text'].apply(lambda x: clean_input(x))
    
        # Remove duplicates
        evidence.drop_duplicates('text', inplace=True)

        # Add summarization
        evidence['summary'] = summarize_text(list(evidence['text'].values))

        # Save for later use
        save_evidence(evidence, claim)
        
    return evidence

In [ ]:
def parse_conclusions(conclusions):
    num_sources = len(conclusions)
    
    false_score = conclusions.count('false')*-1
    unsure_score = conclusions.count('neutral')*-0.1
    true_score = conclusions.count('true')*1
    
    total_score = false_score + unsure_score + true_score
    
    print(f'Conclusions: {conclusions}')
    
#     if total_score < -(num_sources / 3):
#         print('This claim is very likely to be false.')
#     elif total_score < (num_sources / 3):
#         print('This claim is probably untrue.')
#     else:
#         print('This claim is plausible.')

    # rescale score to 0 - 1
    normalized_score = round((10 + total_score) / 20, 2)
    
    print(f'The truth score of this claim is {normalized_score} (scaled between 0 and 1)\n')
    
    return normalized_score

In [ ]:
def clean_input(text):
    # Remove newline
    text = text.replace('\n', ' ')
    return text

def split_to_sentences(text):
    return sent_tokenize(text, language='english')

In [ ]:
def apply_zero_shot_model(claim, text):
    global zero_shot_model
    unsure_threshold = 0.4

    zeroshot_labels = [f"The following statement is True: {claim}",
                       f'Not enough information to verify the following statement: {claim}',
                       f"The following statement is False: {claim}"]

    conclusion = zero_shot_model(text, candidate_labels=zeroshot_labels)
    parsed_conclusion = conclusion['labels'][0].split(' ')[4].lower().replace(':', '')
    if parsed_conclusion == 'true':
        result = True
    elif parsed_conclusion == 'false':
        result = False
    else:
        result = 'neutral'
        
    score = conclusion['scores'][0]
    
    return str(result).lower() if score > unsure_threshold else 'neutral', score

In [ ]:
def apply_nli_model(claim, text):
    global nli_model
    label_mapping = ['false', 'true', 'neutral']   # (['contradiction', 'entailment', 'neutral'])

    scores = nli_model.predict([(claim, text)])
    
    #Convert score to label
    label = label_mapping[scores.argmax(axis=1)[0]]
    confidence = scores[scores.argmax(axis=1)[0]]

    return label, confidence

In [ ]:
def investigate_claim(claim, model_type='nli'):
    
    evidence = collect_evidence(claim, num_results=10)
        
    conclusions = []
    confidences = []
    for i, row in evidence.iterrows():
        source, text, summary = row.source, row.text, row.summary
        
        if model_type == 'zero-shot':
            label, confidence = apply_zero_shot_model(claim, summary)
        elif model_type == 'nli':
            label, confidence = apply_nli_model(claim, summary)
        else:
            raise ValueError('No valid model type specified.')
            
        conclusions += [label]
        confidences += [confidence]

    parse_conclusions(conclusions)
    evidence['conclusion'] = conclusions
    
    return evidence

In [ ]:
def save_evidence(evidence, claim):
    claim = claim.replace(' ', '').strip()
    pd.DataFrame(evidence).to_csv(f"../data/temp/{claim}.csv", sep ='\t')
    
def load_evidence(claim):
    claim = claim.replace(' ', '').strip()
    evidence = pd.read_csv(f"../data/temp/{claim}.csv", sep ='\t')
    return evidence

# Apply model on several claims

## Brad pitt married to Britney Spears? (fake)

In [ ]:
claim = "brad pitt is to marry with britney spears"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

In [ ]:
claim = "brad pitt is not married to britney spears"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

## Joe Biden classified documents found? (true)

In [ ]:
claim = "Joe Biden took home classified documents after leaving the vice-presidency"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

In [ ]:
claim = "Joe Biden never took home any classified documents after leaving the vice-presidency"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

## COVID-19 vaccine causes infertility (fake)

In [ ]:
claim = "COVID-19 vaccine causes infertility"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

In [ ]:
claim = "COVID-19 vaccine does not cause infertility"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

## Jan Smit met de US president (fake, but no relevant evidence)

In [ ]:
claim = "jan smit has met with US president in 2012"
evidence = claim_verification.investigate_claim(claim, model_type='zero-shot')

# Try other models

In [ ]:
from sentence_transformers import CrossEncoder
from transformers import pipeline
import requests
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
import torch

requests.packages.urllib3.disable_warnings() 

import get_evidences

### Setup

In [ ]:
snli_labels = ['contradiction', 'entailment', 'neutral']

In [ ]:
claim = 'COVID-19 vaccine causes infertility'
evidence = claim_verification.load_evidence(claim)
context = evidence.text[0]

### Q&A

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
summarized_context = summarizer(context, min_length=5, max_length=50)
summarized_context = summarized_context[0]['summary_text']
summarized_context

In [ ]:
snli_labels[model.predict([(claim, summarized_context)]).argmax(axis=1)[0]]

In [ ]:
snli_labels[model.predict([claim, test[4]]).argmax()]

In [ ]:
claim = 'COVID-19 vaccine causes infertility'

In [ ]:
test[3]

In [ ]:
model.predict([claim, summarized_context])

In [ ]:
summarized_text = summarizer(context, min_length=10, max_length=128)[0]['summary_text']
sentences = summarized_text.split('.')
sentences = [sent for sent in sentences if len(sent.split(' '))>3]

print(claim)
print(snli_labels[model.predict([(claim, summarized_text)]).argmax(axis=1)[0]]+'\n')
for sent in sentences:
    print(sent)
    print(snli_labels[model.predict([(claim, sent)]).argmax(axis=1)[0]])
    print()

In [ ]:
snli_labels[model.predict([('COVID-19 vaccine does not cause infertility', 'COVID-19 vaccine causes infertility')]).argmax(axis=1)[0]]

In [ ]:
# https://huggingface.co/datasets/snli
model = CrossEncoder('cross-encoder/nli-distilroberta-base')

result = model.predict([(claim, summarized_context)])
print(result)
snli_labels[result.argmax(axis=1)[0]]

### Zero-shot

In [ ]:
from transformers import pipeline

zero_shot_model = pipeline(model="facebook/bart-large-mnli")

In [ ]:
claim1 = "Joe Biden took home classified documents after leaving the vice-presidency"
claim2 = 'documents were found in the home of Biden'

conclusion = oracle(claim2, candidate_labels=[f"{claim1} is true", f"{claim1} is false"])
print(conclusion)

In [ ]:
temp_labels = [f"{claim1} is {str(bool(0))}", f"{claim1} is {str(bool(1))}"]

claim1 = "Joe Biden took home classified documents after leaving the vice-presidency"
claim2 = 'documents were not found in the home of Biden'

conclusion = oracle(claim2, candidate_labels=temp_labels)
bool(np.argmax(conclusion['scores']))

In [ ]:
context = f"{evidence.text[0]}."
conclusion = oracle(context, candidate_labels=[f"{claim} is true", f"{claim} is false"])
print(conclusion)

In [ ]:
context = f"{evidence.text[0]}."
conclusion = oracle(context, candidate_labels=[f"{claim} is false", 
                                               f"{claim} is true"])
print(conclusion)

In [ ]:
context = f"Claim: COVID-19 vaccine does not cause infertility. Information: {summarized_context}."
conclusion = oracle(context, candidate_labels=["False claim", "True claim"])
print(conclusion)

In [ ]:
context = f"Claim: {claim}. Information: {summarized_context}."
conclusion = oracle(context, candidate_labels=["False claim", "True claim"])
print(conclusion)

In [ ]:
# claim = 'COVID-19 vaccine causes infertility'
# for ev in evidence['text'].values:
#     context = f"{claim}. This is logically supported by the following passages. {ev}"
#     conclusion = oracle(context, candidate_labels=["true", "unsure", "false"])
#     print(conclusion)

## Text Generation

In [ ]:
generator = pipeline(model="gpt2", max_length=100)

In [ ]:
generator(f"{summarized_context}. Therefore, is the claim '{claim}' true or false?", do_sample=False)

### Bevindingen

- NLI is te streng